In [1]:
pip install tensorflow keras opencv-python-headless scikit-learn mtcnn


  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/26/d0/22f68eb23eea053a31655960f133c0be9726c6a881547e6e9e7e2a946c4f/opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for mtcnn from https://files.pythonhosted.org/packages/09/d1/2a4269e387edb97484157b872fa8a1953b53dcafbe4842a1967f549ac5ea/mtcnn-0.1.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 3.2 MB/s eta 0:00:13
   ---------------------------------------- 0.5/38.8 MB 9.8 MB/s eta 0:00:04
   - -------------------------------------- 1.7/38.8 MB 18.1 MB/s eta 0:00:03
   --- ------------------------------------ 3.1/38.8 MB 21.8 MB/s eta 0:00:02
   ----- ---------------------------------- 5.3/38.8 MB 28.2 MB/s eta 0:00:02
   ------ --------------------------------- 6.7/38.8 MB 28.5 MB/s eta 0:00:02
   -------- -----------------

In [2]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from mtcnn.mtcnn import MTCNN


In [3]:
# Initialize the MTCNN face detector
detector = MTCNN()

def detect_faces(img):
    # Detect faces in the image
    results = detector.detect_faces(img)
    face_images = []

    for result in results:
        if result['confidence'] >= 0.9:  # Filter by confidence
            x, y, width, height = result['box']
            face_img = img[y:y+height, x:x+width]
            face_images.append(face_img)
    
    return face_images


In [4]:
# Load ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Create a model for feature extraction
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(face_img):
    # Preprocess the image
    face_img = cv2.resize(face_img, (224, 224))
    face_array = image.img_to_array(face_img)
    face_array = np.expand_dims(face_array, axis=0)
    face_array = preprocess_input(face_array)
    
    # Extract features
    features = feature_extractor.predict(face_array)
    return features.flatten()  # Flatten to 1D array


In [5]:
def load_data(dataset_path):
    feature_list = []
    labels = []

    # Iterate through each student directory
    for student in os.listdir(dataset_path):
        student_dir = os.path.join(dataset_path, student)
        if os.path.isdir(student_dir):
            for img_name in os.listdir(student_dir):
                img_path = os.path.join(student_dir, img_name)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                face_images = detect_faces(img)
                for face_img in face_images:
                    features = extract_features(face_img)
                    feature_list.append(features)
                    labels.append(student)

    return np.array(feature_list), np.array(labels)

# Load data
dataset_path = 'dataset_frames'  # Update with your dataset path
X, y = load_data(dataset_path)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━

In [6]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(X_train, y_train)

# Evaluate the classifier
y_pred = svm_classifier.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

      bhavya       1.00      1.00      1.00        22
     nagasai       1.00      1.00      1.00        10
   ramandeep       1.00      1.00      1.00        18
       rehan       1.00      1.00      1.00        17
       ronak       1.00      1.00      1.00        23
     shubham       1.00      1.00      1.00        20
        yash       0.95      1.00      0.97        18
        zoeb       1.00      0.92      0.96        13

    accuracy                           0.99       141
   macro avg       0.99      0.99      0.99       141
weighted avg       0.99      0.99      0.99       141



In [7]:
def predict_face(image_path, svm_classifier, label_encoder, confidence_threshold=0.5):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_images = detect_faces(img)
    for face_img in face_images:
        features = extract_features(face_img)
        probabilities = svm_classifier.predict_proba([features])[0]
        max_index = np.argmax(probabilities)
        confidence = probabilities[max_index]

        if confidence < confidence_threshold:
            print("Predicted: unknown, Confidence:", confidence)
        else:
            label = label_encoder.inverse_transform([max_index])[0]
            print(f"Predicted: {label}, Confidence: {confidence:.2f}")

# Test prediction
test_image_path = 'testdata\Screenshot 2024-08-05 191556.png'
predict_face(test_image_path, svm_classifier, label_encoder)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
Predicted: unknown, Confidence: 0.416310519542512


In [8]:
import joblib

# Save the SVM classifier
svm_classifier_filename = 'svm_classifier.pkl'
joblib.dump(svm_classifier, svm_classifier_filename)

# Save the label encoder
label_encoder_filename = 'label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_filename)


['label_encoder.pkl']

In [9]:
# Save the feature extraction model
feature_extractor.save('feature_extractor.h5')


In [10]:
import joblib
from tensorflow.keras.models import load_model

# Load the SVM classifier
svm_classifier = joblib.load('svm_classifier.pkl')

# Load the label encoder
label_encoder = joblib.load('label_encoder.pkl')

# Load the feature extractor model (if needed)
feature_extractor = load_model('feature_extractor.h5')


In [14]:
def predict_face(image_path, svm_classifier, label_encoder, feature_extractor, confidence_threshold=0.5):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_images = detect_faces(img)
    for face_img in face_images:
        features = extract_features(face_img)
        probabilities = svm_classifier.predict_proba([features])[0]
        max_index = np.argmax(probabilities)
        confidence = probabilities[max_index]

        if confidence < confidence_threshold:
            print("Predicted: unknown, Confidence:", confidence)
        else:
            label = label_encoder.inverse_transform([max_index])[0]
            print(f"Predicted: {label}, Confidence: {confidence:.2f}")

# Load models
svm_classifier = joblib.load('svm_classifier.pkl')
label_encoder = joblib.load('label_encoder.pkl')
# Use feature_extractor from ResNet50 if needed

# Predict with a saved model
predict_face('dataset\bhavya\images\IMG_5278.JPG', svm_classifier, label_encoder, feature_extractor)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
